In [47]:
from keras.layers import Activation, Input, Dropout, merge, Conv2D, Concatenate, tanh
from keras.layers.convolutional import Convolution2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU, ReLU
from keras.activations import tanh
from keras.models import Model

ImportError: cannot import name 'tanh'

In [39]:
def CK(input_layer, filters, k_size=4, strides=2, b_norm=True):
    d = Conv2D(filters, kernel_size=k_size, strides=strides, padding="same")(input_layer)
    print(d)
    if b_norm == True:
        d = BatchNormalization()(d)
        print(d)
    d = LeakyReLU(alpha=0.2)(d)
    print(d)
    print()
    return d

def CDK(input_layer, skip_input, filters, k_size=4, strides=2, b_norm=True) :
    d = UpSampling2D(size=(2, 2))(input_layer)
    print(d)
    d = Conv2D(filters, kernel_size=k_size, padding="same")(d)
    print(d)
    if b_norm == True:
        d = BatchNormalization()(d)
        print(d)
    d = Dropout(0.5)(d)
    print(d)
    d = Concatenate(axis=3)([d, skip_input])
    print(d)
    d = ReLU()(d)
    print(d)
    print()
    return d

In [48]:
input_layer = Input(shape=(1024, 1024, 2))
e1 = CK(input_layer, 64, b_norm=False)
e2 = CK(e1, 128)
e3 = CK(e2, 256)
e4 = CK(e3, 512)
e5 = CK(e4, 512)
e6 = CK(e5, 512)
e7 = CK(e6, 512)
e8 = CK(e7, 512)

d1 = CDK(e8, e7, 512)
d2 = CDK(d1, e6, 1024)
d3 = CDK(d2, e5, 1024)
d4 = CDK(d3, e4, 1024)
d5 = CDK(d4, e3, 1024)
d6 = CDK(d5, e2, 512)
d7 = CDK(d6, e1, 256)

d8 = UpSampling2D(size=(2,2))(d7)
d8 = Conv2D(1, kernel_size=4, padding='same')(d8)
d8 = Activation('tanh')(d8)

Tensor("conv2d_172/BiasAdd:0", shape=(?, 512, 512, 64), dtype=float32)
Tensor("leaky_re_lu_136/LeakyRelu:0", shape=(?, 512, 512, 64), dtype=float32)

Tensor("conv2d_173/BiasAdd:0", shape=(?, 256, 256, 128), dtype=float32)
Tensor("batch_normalization_150/cond/Merge:0", shape=(?, 256, 256, 128), dtype=float32)
Tensor("leaky_re_lu_137/LeakyRelu:0", shape=(?, 256, 256, 128), dtype=float32)

Tensor("conv2d_174/BiasAdd:0", shape=(?, 128, 128, 256), dtype=float32)
Tensor("batch_normalization_151/cond/Merge:0", shape=(?, 128, 128, 256), dtype=float32)
Tensor("leaky_re_lu_138/LeakyRelu:0", shape=(?, 128, 128, 256), dtype=float32)

Tensor("conv2d_175/BiasAdd:0", shape=(?, 64, 64, 512), dtype=float32)
Tensor("batch_normalization_152/cond/Merge:0", shape=(?, 64, 64, 512), dtype=float32)
Tensor("leaky_re_lu_139/LeakyRelu:0", shape=(?, 64, 64, 512), dtype=float32)

Tensor("conv2d_176/BiasAdd:0", shape=(?, 32, 32, 512), dtype=float32)
Tensor("batch_normalization_153/cond/Merge:0", shape=(?, 32, 32, 5

In [49]:
model = Model(input_layer, d8)

In [50]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           (None, 1024, 1024, 2 0                                            
__________________________________________________________________________________________________
conv2d_172 (Conv2D)             (None, 512, 512, 64) 2112        input_20[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_136 (LeakyReLU)     (None, 512, 512, 64) 0           conv2d_172[0][0]                 
__________________________________________________________________________________________________
conv2d_173 (Conv2D)             (None, 256, 256, 128 131200      leaky_re_lu_136[0][0]            
__________________________________________________________________________________________________
batch_norm